# Extract Data

Ноутбук позволяет извлечь данные для создания датасета. В дальнейшем созданный датасет можно использовать при обучении DEM CNN. Для корректной работы нужен ЦММ в формате **GeoTIFF**, а также векторные фигуры в формате **Shapefile**. В фигурах должны быть слои: один с произвольным названием, не представляющим из себя цифру (например, *Segmentation*); остальные с названиями в виде цифр, характеризующих ID класса объекта. В рамках проекта DEM CNN обучен на 10 классах, поэтому необходимы слои с *1* по *10* непосредственно. Названия конкретных фигур не учитываются.

Слой с произвольным названием, *Segmentation*, будет служить для обозначения полезных данных для создания датасета. Всё растровое содержимое вне полигонов этого слоя будет игнорироваться.

Остальные слои должны содержать 4-точечные полигоны объектов непосредственно. Эти полигоны будут аппроксимироваться до горизонтальных ограничивающих рамок.

Датасеты конкатенируются, поэтому ноутбук можно запускать несколько раз на разных ЦММ. При этом уже имеющиеся данные перезаписываются в случае использования одних и тех же параметров.

## Подготовка

Скачиваем необходимые для работы библиотеки. Импортируем их. Задаём пути к ЦММ и векторным фигурам, а также к папке, куда будет сохраняться датасет. Задаём жесткие параметры датасета, рассчитываем окно и подбираем масштабирующие коэффициенты.

In [1]:
%pip install geopandas shapely numpy rasterio tqdm


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from random import shuffle

import geopandas as gpd
from shapely import geometry
import numpy as np
import rasterio
from rasterio.windows import Window
from rasterio.features import rasterize
from tqdm import tqdm

In [3]:
output = os.path.abspath('dataset/')
output_images = os.path.join(output, 'images/')
output_annotations = os.path.join(output, 'annotations/')
if not os.path.exists(output):
    [os.mkdir(dir_) for dir_ in (output, output_images, output_annotations)]
else:
    [os.mkdir(dir_) for dir_ in (output_images, output_annotations) if not os.path.exists(dir_)]
        
train_images, val_images, test_images = [os.path.join(output, 'images/', subset) for subset in ('train/', 'val/', 'test/')]
train_annotations, val_annotations, test_annotations = [os.path.join(output, 'annotations/', subset) for subset in ('train/', 'val/', 'test/')]
[os.mkdir(dir_) for dir_ in (train_images, val_images, test_images, train_annotations, val_annotations, test_annotations) if not os.path.exists(dir_)]

os.path.exists(output)

True

In [4]:
dem = os.path.abspath('dataset/dem.tif')  # it is implied that dem variable will be switched with correct path to GeoTIFF
shapes = os.path.abspath('dataset/shapes.shp')  # it is implied that shapes variable will be switched with correct path to Shapefile
codename = 'query'  # codename for certain DEM to make dataset more distinctive if it was extracted from multiple projects

In [5]:
DESIRED_GSD: float = 0.05  # GSD (m/pixel) that will serve as reference when inferencing a model
INFERENCE_SIZE: int = 640  # size of each image, preferably from 640 or 1280 (the same as the size of input tensor) 
STEP_COEFF: float = 0.66  # coefficient for step size calculation, e.g. 0.66 * 640 = 422 pixels

In [6]:
src = rasterio.open(dem)
gdf = gpd.read_file(shapes)

gdf.drop(gdf.columns.difference(['LAYER', 'geometry']), axis=1, inplace=True)
gdf.rename(columns={'LAYER':'id'}, inplace=True)

objects = gdf[gdf.id.isin([str(i + 1) for i in range(10)])]  # if layer label (ID) is a digit (from 1 up to 10), than it is an object
seg = gdf[~gdf.id.isin([str(i + 1) for i in range(10)])]  # it is a segmentation mask otherwise

In [7]:
if src.crs == {'init': 'EPSG:4326'}:
    lat, lon = src.xy(src.width / 2., src.height / 2.)
    code = int(32700 - round((45. + lon) / 90.) * 100 + round((183. + lat) / 6.))  # UTM zone calculus formula
    l, b, r, t = rasterio.warp.transform_bounds(src.crs, {'init': f'EPSG:{code}'}, *src.bounds)
    GSD = np.average(((r - l) / src.width, (t - b) / src.height))
else:
    GSD = np.average(src.res)

meta = src.meta
meta['height'] = meta['width'] = INFERENCE_SIZE

if GSD > DESIRED_GSD:  # as mentioned, worse GSD than your desired GSD makes dataset suboptimal
    raise ValueError(f'The resolution in the input data {GSD} is too bad')
else:
    print(GSD)

0.041723176431440884


In [8]:
size = int(INFERENCE_SIZE * DESIRED_GSD / GSD)  # window size relative to both desired and real GSD
resize_ratio = INFERENCE_SIZE / size  # to resize annotations pixel-wise coordinates
step = int(STEP_COEFF * size)  # step between windows

steps_x_ = int(np.ceil(src.width / step))
steps_y_ = int(np.ceil(src.height / step))
steps_x = steps_x_ * step
steps_y = steps_y_ * step

size, resize_ratio, step, steps_x, steps_y

(766, 0.835509138381201, 505, 34340, 25755)

## Вспомогательные функции

Загрузка изображения и выжигание полигонов сегментационной маски. Формирование аннотаций на основе координат полигонов.

In [9]:
def get_image(src: rasterio.io.DatasetReader, window: Window, window_polygon: geometry.Polygon, seg: gpd.GeoDataFrame) -> np.ndarray:
    image = src.read(window=window)

    mask = rasterize([intersection for intersection in window_polygon.intersection(seg.geometry) if not intersection.is_empty], 
                     out_shape=image.shape[1:], transform=src.window_transform(window), all_touched=True, dtype=np.float32)
    if not mask.sum() / mask.size >= 0.1:  # sanity check to ignore uninformative DEM regions with area of mask being less than 10%
        return np.zeros_like(image)
    
    image = np.where(mask, image, src.nodata if src.nodata else 0.)

    return image

def get_annotations(polygon: geometry.Polygon, objects: gpd.GeoDataFrame) -> list:
    annotations = []
    for _, object in objects[polygon.intersects(objects.geometry)].iterrows():
        intersection = polygon.intersection(object.geometry)
        x_min, y_max, x_max, y_min = intersection.bounds
        y_min, x_min = src.index(x_min, y_min)
        y_max, x_max = src.index(x_max, y_max)
        x_min, x_max = x_min - x, x_max - x
        y_min, y_max = y_min - y, y_max - y
        
        xyxy = [max(min(coord, size), 0.) * resize_ratio for coord in (x_min, y_min, x_max, y_max)]
        if (xyxy[2] - xyxy[0]) * (xyxy[3] - xyxy[1]) > 100.:  # sanity check to ignore bounding boxes with area less than 100 squared pixels
            annotations.append((object.id, *xyxy))

    return annotations

## Создание датасета

В этом цикле алгоритмом скользящего окна проходимся по ЦММ, находим полезные участки относительно сегментационной маски и создаём на их основе изображения и аннотации. Сохраняем данные в папку для датасета. Участки без полезных данных (или те, где полезных данных меньше 10%) игнорируются скользящим окном (проверка идёт по маске ЦММ, а не растру).

In [10]:
annotated = []  # to store files that have objects in them
unannotated = []  # to sotre files that do not have any objects

pbar = tqdm(total=(steps_x_ - 1) * (steps_y_ - 1))

for y in range(0, steps_y, step):
    for x in range(0, steps_x, step):
        pbar.update()
        
        window = Window(x, y, size, size)
        if not src.read_masks(window=window).any():
            continue

        window_polygon = geometry.Polygon((src.xy(y, x), src.xy(y + size, x), src.xy(y + size, x + size), src.xy(y, x + size), src.xy(y, x)))
        if not seg.geometry.intersects(window_polygon).any():
            continue

        formatted_coords = [str(round(src.xy(y, x)[i], 4)) for i in (0, 1)]

        image = get_image(src, window, window_polygon, seg)
        if not image.any():
            continue

        annotations = get_annotations(window_polygon, objects)

        file = f'{codename}{formatted_coords[0]}_{formatted_coords[1]}_{size}'
        if file not in annotated and file not in unannotated:  # TODO: check why without this statement notebook will fault sometimes
            if annotations:
                annotated.append(file)
            else:
                unannotated.append(file)
        
        image_path = os.path.join(output_images, f'{file}.tif')  # unique name that consists of formatted top left corner coordinates
        with rasterio.open(image_path, 'w', **meta) as dest:
            dest.write(image)

        if annotations:
            annotation_path = os.path.join(output_annotations, f'{file}.txt')  # same unique name as that of the image
            with open(annotation_path, 'w', encoding='utf-8') as file:
                file.writelines(f'{id}\t{x_min}\t{y_min}\t{x_max}\t{y_max}\n' for id, x_min, y_min, x_max, y_max in annotations)

3389it [00:22, 295.51it/s]                          

In [11]:
# src.close()  # uncomment to properly close GeoTIFF just in case it causes errors

## Организация субсетов

Случайно перетасовываем все пары изображение+аннотации, после чего производим разбивку на тренировочный, валидационный и тестировочный субсеты в соотношении 70%, 20% и 10% соответственно. К полученным данным добавляем неаннотированные изображения в размере 40% от субсета (например, при 50 тренировочных аннотированных изображениях к ним добавится 20 неаннотированных). Сортируем субсеты по одноименным директориям. Удаляем лишние неаннотированные изображения.

In [12]:
train_ratio, val_ratio, test_ratio = 0.7, 0.2, 0.1
unannotated_ratio = 0.4

shuffle(annotated)  # optional, comment to leave annotated data as is
shuffle(unannotated)  # optional, comment to leave unannotated data as is

train, val, test = np.split(annotated, [int(len(annotated) * train_ratio), int(len(annotated) * (train_ratio + val_ratio))])
train, val, test = train.tolist(), val.tolist(), test.tolist()

print(len(train), len(val), len(test))

train_len, val_len, test_len = int(unannotated_ratio * len(train)), int(unannotated_ratio * len(val)), int(unannotated_ratio * len(test))
train += unannotated[:train_len]
val += unannotated[train_len + 1:train_len + 1 + val_len]
test += unannotated[train_len + 1 + val_len + 1:train_len + 1 + val_len + 1 + test_len]

print(len(train), len(val), len(test))

9 3 2
12 4 2


In [13]:
for subset, num in ((train, 0), (val, 1), (test, 2)):
    for file in subset:
        image_path = os.path.join(output_images, f'{file}.tif')
        new_image_path = os.path.join(train_images if num == 0 else val_images if num == 1 else test_images, f'{file}.tif')
        os.replace(image_path, new_image_path)
        
        annotation_path = os.path.join(output_annotations, f'{file}.txt')
        if os.path.exists(annotation_path):
            new_annotation_path = os.path.join(train_annotations if num == 0 else val_annotations if num == 1 else test_annotations, f'{file}.txt')
            os.replace(annotation_path, new_annotation_path)

In [14]:
for file in os.listdir(output_images):
    path = os.path.join(output_images, file)
    if os.path.isfile(path):
        os.remove(path)